In [24]:
import tflite_runtime.interpreter as tflite
from keras_image_helper import create_preprocessor
import numpy as np
from io import BytesIO
from urllib import request

from PIL import Image


def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

target_size= 150
    


interpreter = tflite.Interpreter(model_path='dino_dragon-model.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

print(f" input_index is : {input_index}")
print(f" output_index is : {output_index}")


data={ 'url': 'https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg'
    }


target_size= 150

classes = [
    'dino',
    'dragon'
]

def predict(url):
    img=download_image(url)
    img=prepare_image(img, (target_size,target_size))
    x=np.array(img,dtype='float32')
    X=np.array([x])
    X=X/255.0
    X
    
    interpreter.set_tensor(input_index, X)
    interpreter.invoke()
    preds = interpreter.get_tensor(output_index)

    float_predictions = preds[0].tolist()

    return dict(zip(classes, float_predictions))


def lambda_handler(event, context):
    url = event['url']
    result = predict(url)
    return result




 input_index is : 0
 output_index is : 13


In [25]:
data={ 'url': 'https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg'
    }

### Testing lambda_handler function

In [26]:
lambda_handler(data, None)

C:\Users\Asia\AppData\Local\Temp\ipykernel_12488\3173733038.py:21: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.resize(target_size, Image.NEAREST)


{'dino': 0.824486494064331}

### Output of the model dino-vs-dragon-v2.tflite after creating docker image ( Q6)

In [1]:
import requests

url = 'http://localhost:8080/2015-03-31/functions/function/invocations'

data={ 'url': 'https://upload.wikimedia.org/wikipedia/en/e/e9/GodzillaEncounterModel.jpg'
    }
result = requests.post(url, json=data).json()
print(result)

{'dino': 0.31950676441192627}
